# データ可視化のプロセス（前半）：ノーベル賞受賞者のデータ

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shinchu/dataviz-notebooks/blob/main/week_2/nobel-laureates.ipynb)


Wikipediaからスクレイピングしてきたノーベル賞受賞者のデータを使って、データ可視化の6ステップのうち、前半の3ステップをたどります。

1. 問いの設定
2. データの用意
3. データの探索

(---ここまで---)

4. 仮説の設定
5. データの分析
6. データの説明

In [ ]:
# ライブラリをロードする
import pandas as pd
import numpy as np

## 問いの設定

歴代ノーベル賞受賞者の属性に共通するパターンはあるか？

## データの用意

スクレイピングはこの講義のスコープを超えるので、手元にすでにスクレイピング済みのデータがあると仮定します。

PythonによるWebスクレイピングに有用なライブラリとして、下記があります。

* [requests](https://requests-docs-ja.readthedocs.io/en/latest/): HTTP通信用のライブラリ
* [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/): XMLパース用のライブラリ
* [Scrapy](https://scrapy.org/): スクレイピング、クローリング用のフレームワーク

In [ ]:
# JSON形式のファイルを読み込む

file_url = "https://raw.githubusercontent.com/shinchu/dataviz-notebooks/main/data/week_2/nobel_winners_dirty.json"
df = pd.read_json(file_url)

JSONとは、JavaScript Object Notationの略で、以下のような形式のデータです。

```
[
  {
    "born_in": "",
    "category": "Physiology or Medicine",
    "country": "Argentina",
    "date_of_birth": "8 October 1927",
    "date_of_death": "24 March 2002",
    "gender": "male",
    "link": "http://en.wikipedia.org/wiki/C%C3%A9sar_Milstein",
    "name": "César Milstein",
    "place_of_birth": "Bahía Blanca ,  Argentina",
    "place_of_death": "Cambridge , England",
    "text": "César Milstein , Physiology or Medicine, 1984",
    "year": 1984
  },
  {
    "born_in": "Bosnia and Herzegovina",
    "category": "Literature",
    "country": "",
    "date_of_birth": "9 October 1892",
    "date_of_death": "13 March 1975",
    "gender": "male",
    "link": "http://en.wikipedia.org/wiki/Ivo_Andric",
    "name": "Ivo Andric *",
    "place_of_birth": "Dolac (village near Travnik), Austria-Hungary (present-day Bosnia and Herzegovina)",
    "place_of_death": "Belgrade, SR Serbia, SFR Yugoslavia (present-day Serbia)",
    "text": "Ivo Andric *,  born in then  Austria–Hungary , now Bosnia and Herzegovina , Literature, 1961",
    "year": 1961
  }
]
```

### データの確認

In [ ]:
# まずはデータの形をざっと確認

df.head()

In [ ]:
# 列を見る

df.columns

In [ ]:
# 行を見る

df.index

In [ ]:
# 概要を見る

df.info()

ここから、一部のフィールド（性別など）が欠損していることが分かります（1052行のところ、1044要素しかないなど）。

また、読み込んだばかりの状態では、`year`のみが数値（`int64`）になっており、他の変数はオブジェクトとして扱われていることが分かります。オブジェクトはpandasのデフォルトのデータ型で、任意の数値、文字列、日時などを表すことができます。

In [ ]:
# 要約統計量を見る

df.describe()

要約統計量は、デフォルトの状態では数値列だけが表示されます。

最小値は1809ですが、ノーベル賞の授与は1901年から始まったので明らかに正しくない数値が含まれていることが分かります。最大値は2014で、2014年までの受賞者が含まれているようです。

`describe`にパラメータを与えることで、オブジェクトにもアクセスするように指定できます。

In [ ]:
df.describe(include=["object"])

ここから、受賞者には59の国籍があり、米国が350で最大のグループであるなどの情報が得られます。

また、生年月日は全部で1044ありますが、異なり数は853であることが分かります。複数の受賞者が生まれた縁起の良い日があるか、受賞者のデータが重複している可能性があります。

氏名の異なり数が998というのも、受賞者のデータが重複している可能性を示すものです。2つ以上の賞を受賞している人は数名いますが（調べた限りでは4名）、54の重複はそれに対して多すぎます。

[Multiple Nobel Prize laureates](https://www.nobelprize.org/prizes/facts/nobel-prize-facts/#:~:text=Linus%20Pauling%20is%20the%20only,the%201962%20Nobel%20Peace%20Prize.)

In [ ]:
# もう一度、生データを確認

df.head(10)

In [ ]:
df.tail(5)

よく見ると、名前の後ろにアスタリスクがついている人がいたり、生年月日のフォーマットが統一されていなかったり、`born_in`の情報がほぼ存在しない、といったことが分かります。

ここまでの情報をもとに、現時点でのデータ仕様書を作っておきましょう。

前処理が終わった後にデータ仕様書をアップデートします。

In [ ]:
# 現時点のデータ仕様書を作る

| カラム | カラムの説明 | データ型 | データ型（理想） | 尺度 | 値の説明 | NULL | UNIQ | CHECK |
|:--|:--|:--|:--|:--|:--|:--|:--|:--|
| born_in | 出生地 | テキスト | カテゴリ | 名義尺度 |  | ✔ |  |  |
| category | 賞の種類 | テキスト | カテゴリ | 名義尺度 |  | ✔ |  | | 
| country | 国籍 | テキスト | カテゴリ | 名義尺度 |  | ✔ |  |  |
| date_of_birth | 生年月日 | テキスト | 数値 | 間隔尺度 |  | ✔ |  |  |
| date_of_death | 没年月日 | テキスト | 数値 | 間隔尺度 |  | ✔ |  |  |
| gender | 性別 | テキスト | カテゴリ | 名義尺度 |  | ✔ |  |  |
| link | ウィキペディアへのリンク | テキスト | テキスト | テキスト |  | ✔ |  |  |
| name | 氏名 | テキスト | テキスト | テキスト |  | ✔ |  |  |
| place_of_birth | 出生地 | テキスト | カテゴリ | 名義尺度 |  | ✔ |  |  |
| place_of_death | 死没地 | テキスト | カテゴリ | 名義尺度 |  | ✔ |  |  |
| text | 説明 | テキスト | テキスト | テキスト |  | ✔ |  |  |
| year | 受賞年 | 数値 | 数値 | 間隔尺度 |  | ✔ |  | 1901 $\leq$ x $\leq$ 2022 |

* `NULL`: 欠損値が存在する可能性があるか
* `UNIQ`: 重複しない値か
* `CHECK`: 満たすべき条件

In [ ]:
# データの前処理（欠損値、重複、整列されていない行、不統一、エラー値、データの混合などに対処する）

In [ ]:
# born_in

まずは、ほぼ存在しない`born_in`を適切に欠損値として処理できるようにします。

`apply`メソッドを使うと、セルに対して同じ処理を繰り返し実行できます。これを使って`born_in`の実際のデータ型（pandasのオブジェクトにラップされているもの）を見てみましょう。

In [ ]:
df["born_in"].apply(type)

`set`を使って集合をとることで、異なり値だけを取り出します。

In [ ]:
set(df["born_in"].apply(type))

すべてのデータがテキスト型であることが分かりました。何もないように見えるセルにも空白文字（スペース、タブ、改行など）が入っているようです。

空白文字を欠損値（`NaN`）に置換しましょう。

In [ ]:
df["born_in"].replace("", np.nan, inplace=True)

In [ ]:
df["born_in"].count()

これで空白文字が欠損値に置換され、`born_in`の正しい数が分かるようになりました。

同じ要領で、データセット内のすべての空文字列を欠損値に置換します。

In [ ]:
df.replace("", np.nan, inplace=True)

In [ ]:
# name

さきほど、アスタリスクが付いた名前がありました。これを整えていきます。

まず、アスタリスクが付いた名前の数を調べます。

In [ ]:
set(df["name"].apply(type))

`name`も文字列ということが分かります。

`name`に`*`が含まれているセルを`str.contains()`で見てみます。

`\*`という風にアスタリスクの前にバックスラッシュをつけるのは、アスタリスクを正規表現の記号（直前の文字の0回以上の繰り返し）ではなく、通常の文字としてのアスタリスクとして扱うためです。これを「エスケープする」と言います。

[正規表現](https://ja.wikipedia.org/wiki/%E6%AD%A3%E8%A6%8F%E8%A1%A8%E7%8F%BE)

In [ ]:
df["name"].str.contains("\*")

アスタリスクが付いている名前が`True`判定されています。

これをデータフレームに代入し、`name`列だけを取り出すことができます。

In [ ]:
df[df["name"].str.contains("\*")]["name"]

142個ありました。

把握できたところで、アスタリスクを置換し、前後の空白も取ります。

置換には`str.replace()`、前後の空白削除には`str.strip()`を使います。

In [ ]:
df["name"] = df["name"].str.replace('*', '', regex=False)

In [ ]:
df["name"] = df["name"].str.strip()

In [ ]:
df[df.name.str.contains("\*")]["name"]

再度数えてみると、アスタリスク付きの名前がなくなっていることが分かります。

In [ ]:
# 重複

次に、重複しているデータに対処します。`duplicated`メソッドで重複している行が`True`と判定されます。

In [ ]:
df.duplicated("name", keep=False)

これをデータセット全体の条件にすると、氏名が重複している行のみを抽出できます。

In [ ]:
all_dupes = df[df.duplicated("name", keep=False)]

In [ ]:
all_dupes

299行の重複があることが分かりました。重複行を名前で並べ替え、国籍と受賞年を見てみます。

In [ ]:
all_dupes.sort_values("name")[["born_in", "name", "country", "year"]]

同じ年の受賞で異なる国籍になっている人がいますし、国籍が欠損している場合もあります。

今回参照したWikipediaでは、国ごとに受賞者のリストが作られているので、受賞者が別の国に移住した場合などは移住前後の国の両方が書かれているようです。

また、国籍が欠損してる場合は、`born_in`に国名が入っているようです。

ここではまず、`born_in`の情報を失くさないために同一受賞者の欠損箇所にコピーします。

In [ ]:
pd.set_option('display.max_rows', 300)

`born_in`が欠損していない行のリストを作ります。

In [ ]:
bi_df = all_dupes[~all_dupes["born_in"].isna()]

In [ ]:
bi_df.head()

bi_dfに含まれている受賞者を全体から取り出し、`name`と`born_in`で並べ替えます。こうすることで、必ず同一人物の`born_in`が欠損しているデータが欠損していないデータのすぐ後にくることになります。

In [ ]:
df[["born_in", "name"]][df["name"].isin(bi_df["name"])].sort_values(["name", "born_in"])

`fillna(method="ffill")`で、直前の値で欠損値を埋めます。

In [ ]:
df[["born_in", "name"]][df["name"].isin(bi_df["name"])].sort_values(["name", "born_in"]).fillna(method="ffill")

再度インデックス順に並べ替え、元のデータフレームに代入します。

In [ ]:
df.loc[df[df["name"].isin(bi_df["name"])].index, ["born_in", "name"]] = df[["born_in", "name"]][df["name"].isin(bi_df["name"])].sort_values(["name", "born_in"]).fillna(method="ffill").sort_index()

In [ ]:
df.describe(include=["object"])

`born_in`の値が250個になりました。

`all_dupes`を作り直します。

In [ ]:
all_dupes = df[df.duplicated("name", keep=False)]

In [ ]:
all_dupes.sort_values(["name", "born_in"])[["born_in", "name", "country", "year"]]

`all_dupes`のうち、国籍が欠損しているデータは削除しても構わないようになったので、削除します。

In [ ]:
remove_index = all_dupes[all_dupes["country"].isna()].index

In [ ]:
df = df.drop(remove_index)

In [ ]:
df.describe(include=["object"])

受賞者数が941人になりました。まだ重複が国籍違いによる重複が解消されていないためです。

In [ ]:
all_dupes = df[df.duplicated("name", keep=False)]

In [ ]:
all_dupes.sort_values(["name", "born_in"])[["born_in", "name", "country", "year"]]

また、よく見ると

* キュリー夫人の名前が2種類（Marie Curie、Marie Skłodowska-Curie）ある
* Ragnar Granitが1809年に受賞されたことになっている（ノーベル賞は1901年開始）
* Sidney Altmanは一度しか受賞されていないはずなのに、受賞年がずれている

ことが分かります。

ここは一つずつ修正します。

In [ ]:
df.drop(df[df["year"] == 1809].index, inplace=True)

In [ ]:
df = df[~(df["name"] == "Marie Curie")]

In [ ]:
df.loc[(df["name"] == "Marie Skłodowska-Curie") & (df["year"] == 1911), "country"] = "France"

In [ ]:
df = df[~((df["name"] == "Sidney Altman") & (df["year"] == 1990))]

In [ ]:
all_dupes = df[df.duplicated("name", keep=False)]

In [ ]:
all_dupes.sort_values(["name", "born_in"])[["born_in", "name", "country", "year"]]

残りの重複については、基準を決めて調査してどのデータを残すのかを決めることもできますが、ここでは簡便のために確率的な方法をとります。

つまり、重複しているデータの中から無作為に半分を選び取り、残りの半分を削除します。

まず、インデックスをランダムにシャッフルします。

In [ ]:
df = df.reindex(np.random.permutation(df.index))

In [ ]:
df

次に、`drop_duplicates`で重複データのうち、最初に出現したものだけを残します。

In [ ]:
df = df.drop_duplicates(["name", "year"])

再度インデックスを順番に並べます。

In [ ]:
df = df.sort_index()

In [ ]:
all_dupes = df[df.duplicated("name", keep=False)]

In [ ]:
all_dupes.sort_values(["name", "year"])[["born_in", "name", "country", "year", "category"]]

重複が正しく、実際に2回受賞した人のみになりました。

In [ ]:
# 更に欠損値の処理

In [ ]:
len(df)

In [ ]:
df.count()

カテゴリがないデータが3つあるようです。

ちなみに、カテゴリの名称はこちら。

In [ ]:
df["category"].value_counts()

In [ ]:
df[df["category"].isnull()]

手動で補完します。

In [ ]:
df.loc[812, "category"] = "Physiology or Medicine"

In [ ]:
df.loc[815, "category"] = "Economics"

In [ ]:
df.loc[922, "category"] = "Physiology or Medicine"

In [ ]:
df.count()

更に、ジェンダーも数人欠けています。

In [ ]:
df[df["gender"].isnull()]["name"]

これを見ると、Ragnar Granit以外は団体です。今回は個人に着目したいので、団体の受賞者は削除することにします。

In [ ]:
df.loc[650, "gender"] = "male"

In [ ]:
df = df[df["gender"].notnull()]

In [ ]:
df.count()

欠けているデータをできる限り補完してみます。

In [ ]:
df[df["date_of_birth"].isnull()]["name"]

In [ ]:
df.loc[782, "date_of_birth"] = "11 September 1960"

In [ ]:
df[df["country"].isnull()]

`country`は欠損値が多いので個別に埋めることはせず、`born_in`を持ってくることにします。

In [ ]:
df.loc[df["country"].isnull(), "country"] = df[df["country"].isnull()]["born_in"]

In [ ]:
df.count()

これで、`category`、`date_of_birth`、`gender`、`country`、`year`、`text`等主要なデータが揃っているクリーンなデータができました。

また、tidy dataにもなっています。

最後に日付のフォーマットを整えます。

In [ ]:
# 日付

In [ ]:
df["date_of_birth"] = pd.to_datetime(df["date_of_birth"], errors="raise")

In [ ]:
pd.to_datetime(df.date_of_death, errors="raise")

生年月日は無事に変換できましたが、没年月日ではエラーが出ました。

エラーを捕捉して修正します。

In [ ]:
for i, row in df.iterrows():
    try:
        pd.to_datetime(row.date_of_death, errors="raise")
    except:
        print(f"{row.date_of_death.ljust(30)} ({row['name']}, {i})")

エラーが発生している箇所で、日付の様々なフォーマットが確認できます。

手動で修正することもできますが、ここでは一律で欠損値に変換したいと思います。

In [ ]:
df["date_of_death"] = pd.to_datetime(df.date_of_death, errors="coerce")

日付をきちんと変換できたので、ノーベル賞受賞時の受賞者の年齢を計算してみます。

In [ ]:
df["award_age"] = df.year - pd.DatetimeIndex(df["date_of_birth"]).year

気になるので、最年少と最高齢受賞者を見てみましょう。

In [ ]:
df.sort_values("award_age")[["name", "award_age", "year", "category"]]

最年少はマララさんですね。比較的最近です。

その次は比較的早い時代の、ローレンス・ブラッグ。受賞理由は「X線による結晶構造解析に関する研究」、現代結晶学の創始者だそうです。

最高齢は経済学者のレオニード・ハーヴィッツ。「メカニズムデザインの理論の基礎を確立した功績を称えて」とのこと。

ともかくこれで、分析しやすいキレイなデータができました。

ファイルに保存しておきます。元のデータがローカルにある時は、フォルダを分けて保存すると良いでしょう。

In [ ]:
df.to_csv("nobel_prize_data_cleaned.tsv", index=False, header=True, sep="\t")

In [ ]:
# Colabからダウンロードする

from google.colab import files

files.download("nobel_prize_data_cleaned.tsv")

In [ ]:
# 最終的なデータ仕様書

| カラム | カラムの説明 | データ型 | 尺度 | 値の説明 | NULL | UNIQ | CHECK |
| :-- | :-- | :-- | :-- | :-- | --- | --- | --- |
| born_in | 出生地 | カテゴリ | 名義尺度 |  | ✔ |  |  |
| category | 賞の種類 | カテゴリ | 名義尺度 |  |  |  |  |
| country | 国籍 | カテゴリ | 名義尺度 |  |  |  |  |
| date_of_birth | 生年月日 | 数値 | 間隔尺度 |  |  |  |  |
| date_of_death | 没年月日 | 数値 | 間隔尺度 |  | ✔ |  |  |
| gender | 性別 | カテゴリ | 名義尺度 |  |  |  |  |
| link | ウィキペディアへのリンク | テキスト | テキスト |  |  |  |  |
| name | 氏名 | テキスト | テキスト |  |  |  |  |
| place_of_birth | 出生地 | カテゴリ | 名義尺度 |  | ✔ |  |  |
| place_of_death | 死没地 | カテゴリ | 名義尺度 |  | ✔ |  |  |
| text | 説明 | テキスト | テキスト |  |  |  |  |
| year | 受賞年 | 数値 | 間隔尺度 |  |  |  | 1901 $\leq$ x $\leq$ 2014 |
| award_age | 受賞年齢 | 数値 | 比例尺度 |  |  |  |  |


* `NULL`: 欠損値が存在するか
* `UNIQ`: 重複しない値か
* `CHECK`: 満たすべき条件

## データの探索（探索的可視化）

お気づきのように、データの前処理の中である程度データを探索してきました。

続けて、いくつかの気になった点について探索的可視化を行い、最初に立てた「問い」に基づいてどのような仮説が設定できそうかを探っていきます。


どこから着手してよいかわからない時は、講義で話した6つのアプローチを出発点にしてみましょう。

1. データの分布を見る
2. データの関係を見る
3. データを縮約する
4. データを層別にする
5. データを詳細化する
6. データを時系列で見る


後ほどの演習で練習する、Matplotlib（とそのラッパーのseaborn）というライブラリを使います。

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# seabornの見た目を使う

sns.set()

### 男女間の差

実はpandasはplotメソッドでmatplotlibを呼び出すことができるため、自分で見るための図は簡単に作れます。

In [ ]:
by_gender = df.groupby("gender")
by_gender.size().plot(kind="bar")

In [ ]:
by_gender.size()

In [ ]:
by_cat_gen = df.groupby(["category", "gender"])

In [ ]:
by_cat_gen.size()

In [ ]:
by_cat_gen.size().plot(kind="barh")

データを横持ちにして再度プロットしてみます。

In [ ]:
by_cat_gen.size().unstack()

In [ ]:
by_cat_gen.size().unstack().plot(kind="barh")

受賞者の合計を計算し、女性受賞者の人数順に並べ替えます。

In [ ]:
cat_gen_sz = by_cat_gen.size().unstack()
cat_gen_sz["total"] = cat_gen_sz.sum(axis=1)
cat_gen_sz = cat_gen_sz.sort_values(by="female", ascending=True)
cat_gen_sz[["female", "total", "male"]].plot(kind="barh")

In [ ]:
df[(df.category == 'Physics') & (df.gender == 'female')][['name', 'country','year']]

[マリア・ゲッパート＝メイヤー](https://ja.wikipedia.org/wiki/%E3%83%9E%E3%83%AA%E3%82%A2%E3%83%BB%E3%82%B2%E3%83%83%E3%83%91%E3%83%BC%E3%83%88%EF%BC%9D%E3%83%A1%E3%82%A4%E3%83%A4%E3%83%BC)

「原子核の殻構造に関する研究」

ちょっと予告。インタラクティブな図を作ると、細かい数値が見やすい。

In [ ]:
!pip install plotly

In [ ]:
import plotly.express as px
from plotly import offline

In [ ]:
count_df = cat_gen_sz.stack().reset_index()

In [ ]:
count_df = count_df.rename(columns={0: "count"})

In [ ]:
count_df

Tidy dataにしてから使う。

In [ ]:
fig = px.bar(count_df, x="count", y="category", color="gender", barmode="group")
offline.iplot(fig)

### 歴史的傾向

例えば、女性の受賞者は時間の推移によって増加しているのでしょうか？時系列にプロットすることで見えてきそうです。

In [ ]:
by_year_gender = df.groupby(["year", "gender"])
year_gen_sz = by_year_gender.size().unstack()
year_gen_sz.plot(kind="bar", figsize=(16, 4))

だいぶ見づらいので、x軸のラベルを調整します。

In [ ]:
def thin_xticks(ax, tick_gap=10, rotation=45):
    ticks = ax.xaxis.get_ticklocs()
    ticklabels = [l.get_text() for l in ax.xaxis.get_ticklabels()]
    ax.xaxis.set_ticks(ticks[::tick_gap])
    ax.xaxis.set_ticklabels(ticklabels[::tick_gap], rotation=rotation)

    ax.figure.show()

また、欠損している受賞年（1939–1945年の第二次世界大戦中はノーベル賞が授与されませんでした）は飛ばすのではなく、欠損として表示したいと思います。

indexを振り直します。

In [ ]:
new_index = pd.Index(np.arange(1901, 2015), name="year")
by_year_gender = df.groupby(["year", "gender"])
year_gen_sz = by_year_gender.size().unstack().reindex(new_index)

In [ ]:
year_gen_sz

更に、見やすくするために図を上下に並べて表示することにします。これにはMatplotlibのサブプロットという機能を使います。

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, sharex=True, sharey=True)

ax_f = axes[0]
ax_m = axes[1]

fig.suptitle("Nobel Prize laureates by gender", fontsize=16)

ax_f.bar(year_gen_sz.index, year_gen_sz.female)
ax_f.set_ylabel("Female laureates")

ax_m.bar(year_gen_sz.index, year_gen_sz.male)
ax_m.set_ylabel("Male laureates")

ax_m.set_xlabel("Year")

女性の受賞者が近年増えているような気もしますが、受賞者全体が増加傾向にあるようにも見えます。共同受賞する人数が増えているためでしょうか？

これについては、例えば「知識量が爆発的に増え、研究者間・分野間の協力が必須となった現代ではノーベル賞の同時受賞人数が増えている」などと仮説を設定していろいろと探索できそうです。

更に、このデータに関しては、

* 国別の傾向
    * 人口1人当たりの受賞者数
    * 分野別の受賞数
    * 受賞分布の歴史的傾向
* 受賞者の年齢と没年齢
* 受賞者の移住

など、様々な側面で探索できそうです。

興味があったら問いを立てて、探索してみてください。

このデータセットは今後講義でも使っていきます。